In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data = pd.read_csv('../input/credit-card-customers/BankChurners.csv')

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
data.drop(columns=['CLIENTNUM','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],inplace=True)

In [ ]:
Attrition_Flag_map = {'Existing Customer':1,'Attrited Customer':0}
data['Attrition_Flag'] = data['Attrition_Flag'].map(Attrition_Flag_map)

In [ ]:
data = pd.get_dummies(data,columns=['Gender','Marital_Status','Card_Category',],drop_first=True)

In [ ]:
Education_Level_map = {'Uneducated':0,'Unknown':1,'High School':2,'College':3,'Graduate':4,'Post-Graduate':5,'Doctorate':6}
data['Education_Level'] = data['Education_Level'].map(Education_Level_map)

In [ ]:
salary = []
for i in data['Income_Category']:
    try:
        salary.append(re.findall(r'\d+(?=K)',i)[0])
    except:
        salary.append(i)
salary_series = pd.Series(salary)
data['Income_Category'] = salary_series
    

In [ ]:
for i in data.columns:
    data[i] = pd.to_numeric(data[i],errors='coerce')

In [ ]:
data['Income_Category'] = data['Income_Category'].fillna(0)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
for i in data.columns:
    try:
        sns.boxplot(data[i])
        plt.show()
    except:
        print(i)

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
x = data.drop(columns=['Attrition_Flag'])
y = data['Attrition_Flag']

In [ ]:
from imblearn.over_sampling import SMOTE 

In [ ]:
sm = SMOTE(random_state=42)
x_res, y_res = sm.fit_resample(x, y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_res,y_res,test_size=0.20,random_state=71,stratify=y_res)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_scale = MinMaxScaler()
test_scale = MinMaxScaler()

In [ ]:
x_train_scaled = train_scale.fit_transform(x_train)
x_test_scaled = train_scale.fit_transform(x_test)

In [ ]:
x_train_scaled.shape

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape = (23,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(5000,activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(5000,activation='relu'),
    keras.layers.Dense(1500,activation='relu'),
    keras.layers.Dense(1500,activation='relu'),
    keras.layers.Dense(500,activation='relu'),
    keras.layers.Dense(500,activation='relu'),
    keras.layers.Dense(250,activation='relu'),
    keras.layers.Dense(250,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')

In [ ]:
model.fit(x_train_scaled,y_train,epochs=250,batch_size=64)

In [ ]:
model.evaluate(x_test_scaled,y_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = np.round(model.predict(x_test_scaled))

In [ ]:
print(classification_report(y_pred=y_pred,y_true=np.array(y_test)))